In [34]:
import pennylane as qml
from pennylane import numpy as np
from tqdm import tqdm

from QFM import QFM

random_seed = 42
np.random.seed(random_seed)

# ====== 构造测试数据 ====== 
# 两个 2-qubit 的态：|00> 和 |11>
num_samples = 10
n_features = 2**2  # 4

# CNOT matrix for 2-qubit system
CNOT = np.array([[1, 0, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 0, 1],
                    [0, 0, 1, 0]])

# 输入态：随机扰动的 |01> + |10>
input_states = []
for _ in range(num_samples):
    state = np.array([0, 1, 1, 0], dtype=float) + 0.01 * np.random.randn(4)
    state = state / np.linalg.norm(state)
    input_states.append(state)
input_states = np.array(input_states)

# 输出态：CNOT 作用后的态
output_states = []
for _ in range(num_samples):
    state = input_states[_]
    state = CNOT @ state
    output_states.append(state)
output_states = np.array(output_states)

print("Input shape:", input_states.shape)
print("Output shape:", output_states.shape)

# ====== 实例化并训练 ====== 
model = QFM(
    input_samples=input_states,
    output_samples=output_states,
    n_ancilla=2,
    num_time_steps=4,
    depth_per_time_step=2
)

model.fit(epochs=32, batch_size=2, learning_rate=0.05)

# ====== 测试推理 ======
x_test = np.mean(input_states, axis=0)
y_test = np.mean(output_states, axis=0)

print("\nTest swap test value:", model.circuit(model.parameters, x_test, y_test))


Input shape: (10, 4)
Output shape: (10, 4)
Training time step 1/4


Cost after time step 1: 0.5001024999003144
Training time step 2/4


Cost after time step 2: 0.007354140506856122
Training time step 3/4


Cost after time step 3: 0.007663404707833806
Training time step 4/4


Cost after time step 4: 6.312688465903996e-05

Test swap test value: 0.9999368731153406
